# Αναγνώριση Προτύπων

### 3η Εργαστηριακή Άσκηση
### Αναγνώριση Είδους και Εξαγωγή Συναισθήματος από Μουσική

Ακαρέπης Ανδρέας, 03117058

Κόντη Πολυξένη Ειρήνη, 03117180

In [ ]:
import numpy as np
import copy
import re
import os
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, recall_score, classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import SubsetRandomSampler, DataLoader
import torch.nn.functional as F
from torchvision import transforms
import torchvision.models as models
from scipy.stats import spearmanr
from IPython.display import Image

In [ ]:
os.listdir("../input/patreco3-multitask-affective-music/data/")

###Βήμα 1

Επιλέγονται τα αρχεία '65784.beatsync.fused.npy.gz' και '80238.beatsync.fused.npy.gz' που ανήκουν στις κατηγορίες classical και blues αντίστοιχα

In [ ]:
#β
specClassical = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/train/65784.fused.full.npy')
specBlues = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/train/80238.fused.full.npy')

In [ ]:
print(specClassical.shape)
print(specBlues.shape)

In [ ]:
melClassical, chromaClassical = specClassical[:128], specClassical[128:]
melBlues, chromaBlues = specBlues[:128], specBlues[128:]

In [ ]:
#γ
fig, ax = plt.subplots()
img = librosa.display.specshow(melClassical, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Spectrogram of a Classical type')
fig.colorbar(img, ax=ax, format="%+2.f dB")


fig, ax = plt.subplots()
img = librosa.display.specshow(melBlues, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Spectrogram of an Blues type')
fig.colorbar(img, ax=ax, format="%+2.f dB")

### Βήμα 2

In [ ]:
#α
print(melClassical.shape)
print(melBlues.shape)

In [ ]:
#β
specClassicalBeat = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train/65784.fused.full.npy')
specBluesBeat = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train/80238.fused.full.npy')

In [ ]:
print(specClassicalBeat.shape)
print(specBluesBeat.shape)

In [ ]:
melClassicalBeat, chromaClassicalBeat = specClassicalBeat[:128], specClassicalBeat[128:]
melBluesBeat, chromaBluesBeat = specBluesBeat[:128], specBluesBeat[128:]

In [ ]:
fig, ax = plt.subplots()
img = librosa.display.specshow(melClassicalBeat, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Spectrogram of a Classical type')
fig.colorbar(img, ax=ax, format="%+2.f dB")


fig, ax = plt.subplots()
img = librosa.display.specshow(melBluesBeat, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Spectrogram of an Blues type')
fig.colorbar(img, ax=ax, format="%+2.f dB")

### Βήμα 3

In [ ]:
print(chromaClassical.shape)
print(chromaBlues.shape)

print(chromaClassicalBeat.shape)
print(chromaBluesBeat.shape)

In [ ]:
fig, ax = plt.subplots()
img = librosa.display.specshow(chromaClassical, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Chromagram of a Classical type')
fig.colorbar(img, ax=ax, format="%+2.f dB")


fig, ax = plt.subplots()
img = librosa.display.specshow(chromaBlues, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Chromagram of an Blues type')
fig.colorbar(img, ax=ax, format="%+2.f dB")

In [ ]:
fig, ax = plt.subplots()
img = librosa.display.specshow(chromaClassicalBeat, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Chromagram of a Classical type')
fig.colorbar(img, ax=ax, format="%+2.f dB")


fig, ax = plt.subplots()
img = librosa.display.specshow(chromaBluesBeat, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Chromagram of an Blues type')
fig.colorbar(img, ax=ax, format="%+2.f dB")

### Βήμα 4

In [ ]:
import copy
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

# HINT: Use this class mapping to merge similar classes and ignore classes that do not work very well
CLASS_MAPPING = {
    "Rock": "Rock",
    "Psych-Rock": "Rock",
    "Indie-Rock": None,
    "Post-Rock": "Rock",
    "Psych-Folk": "Folk",
    "Folk": "Folk",
    "Metal": "Metal",
    "Punk": "Metal",
    "Post-Punk": None,
    "Trip-Hop": "Trip-Hop",
    "Pop": "Pop",
    "Electronic": "Electronic",
    "Hip-Hop": "Hip-Hop",
    "Classical": "Classical",
    "Blues": "Blues",
    "Chiptune": "Electronic",
    "Jazz": "Jazz",
    "Soundtrack": None,
    "International": None,
    "Old-Time": None,
}


def torch_train_val_split(
    dataset, batch_train, batch_eval, val_size=0.2, shuffle=True, seed=420
):
    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    val_split = int(np.floor(val_size * dataset_size))
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(indices)
    train_indices = indices[val_split:]
    val_indices = indices[:val_split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=batch_train, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=batch_eval, sampler=val_sampler)
    return train_loader, val_loader


def read_spectrogram(spectrogram_file, chroma=False, mel = False):
    # with open(spectrogram_file, "r") as f:
    if chroma == False and mel == False:
        spectrograms = np.load(spectrogram_file)
    elif mel == True:
        spectrograms = np.load(spectrogram_file)[:128]
    else:
        spectrograms = np.load(spectrogram_file)[128:]
    # spectrograms contains a fused mel spectrogram and chromagram
    # Decompose as follows
    return spectrograms.T


class LabelTransformer(LabelEncoder):
    def inverse(self, y):
        try:
            return super(LabelTransformer, self).inverse_transform(y)
        except:
            return super(LabelTransformer, self).inverse_transform([y])

    def transform(self, y):
        try:
            return super(LabelTransformer, self).transform(y)
        except:
            return super(LabelTransformer, self).transform([y])


class PaddingTransform(object):
    def __init__(self, max_length, padding_value=0):
        self.max_length = max_length
        self.padding_value = padding_value

    def __call__(self, s):
        if len(s) == self.max_length:
            return s

        if len(s) > self.max_length:
            return s[: self.max_length]

        if len(s) < self.max_length:
            s1 = copy.deepcopy(s)
            pad = np.zeros((self.max_length - s.shape[0], s.shape[1]), dtype=np.float32)
            s1 = np.vstack((s1, pad))
            return s1


class SpectrogramDataset(Dataset):
    def __init__(
        self, path, class_mapping=None, train=True, max_length=-1, regression=None, chroma = False, mel = False
    ):
        t = "train" if train else "test"
        p = os.path.join(path, t)
        self.regression = regression

        self.index = os.path.join(path, "{}_labels.txt".format(t))
        self.files, labels = self.get_files_labels(self.index, class_mapping)
        self.feats = [read_spectrogram(os.path.join(p, f), chroma, mel) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length)
        self.label_transformer = LabelTransformer()
        if isinstance(labels, (list, tuple)):
            if not regression:
                self.labels = np.array(
                    self.label_transformer.fit_transform(labels)
                ).astype("int64")
            else:
                self.labels = np.array(labels).astype("float64")

    def get_files_labels(self, txt, class_mapping):
        # Returns a list of file names and a list of their labels
        with open(txt, 'r') as fd:
            lines = [l.rstrip().split('\t') for l in fd.readlines()[1:]]
        files, labels = [], []
        for l in lines:
            label = l[1]
            if class_mapping:
                label = class_mapping[l[1]]
            if not label:
                continue
            # Kaggle automatically unzips the npy.gz format so this hack is needed
            _id = l[0].split('.')[0]
            npy_file = '{}.fused.full.npy'.format(_id)
            files.append(npy_file)
            labels.append(label)
            
        return files, labels

    def __getitem__(self, item):
        length = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], length

    def __len__(self):
        return len(self.labels)


if __name__ == "__main__":
    ##################################################################################
    # load fused speectrogram + chromagram for the full (non-beat-synced) data
    ##################################################################################
    specs_fused = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/', train=True,
                                     class_mapping=CLASS_MAPPING, max_length=-1)
    train_loader, val_loader = torch_train_val_split(specs_fused, 32, 32, val_size=.33)
    test_dataset = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/', train=False,
                                     class_mapping=CLASS_MAPPING, max_length=-1)
    test_loader = DataLoader(test_dataset, batch_size=1)
    
    ##################################################################################
    # load single synced mel spectrograms
    ##################################################################################
    # Dataset
    mel_specs = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/', train=True,
                                        class_mapping=CLASS_MAPPING, max_length=-1,mel = True)
    # Train and Test loaders
    train_loader_mel, val_loader_mel = torch_train_val_split(mel_specs, 32, 32, val_size=.33)
    test_dataset_mel = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/', train=False,
                                            class_mapping=CLASS_MAPPING, max_length=-1,mel = True)
    
    test_loader_mel = DataLoader(test_dataset_mel, batch_size=1)
    
    ##################################################################################
    # load beat synced mel spectrograms
    ##################################################################################
    # Dataset
    beat_mel_specs = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/', train=True,
                                         class_mapping=CLASS_MAPPING, max_length=-1,mel = True)
    # Train and Test loaders
    train_loader_beat_mel, val_loader_beat_mel = torch_train_val_split(beat_mel_specs, 32, 32, val_size=.33)
    test_dataset_beat_mel = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/', train=False,
                                                 class_mapping=CLASS_MAPPING, max_length=-1,mel = True)
    test_loader_beat_mel = DataLoader(test_dataset_beat_mel, batch_size=1)
    
    ##################################################################################
    # load beat synced chroma chromagrams
    ##################################################################################
    beat_chroma = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/', train=True,
                                     class_mapping=CLASS_MAPPING, max_length=-1,chroma = True)
    train_loader_beat_chroma, val_loader_beat_chroma = torch_train_val_split(beat_chroma, 32, 32, val_size=.33)
    test_dataset_beat_chroma = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/', train=False,
                                                 class_mapping=CLASS_MAPPING, max_length=-1,chroma = True)
    test_loader_beat_chroma = DataLoader(test_dataset_beat_chroma, batch_size=1)

    


**Πριν την ομαδοποίηση των κλάσεων**

In [ ]:
y_train = []
y_test = []

# Train labels
file = open('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train_labels.txt', 'r') 
for line in file.readlines()[1:]: 
    label = line.split()[1]
    y_train.append(label)
    
# Test labels
file = open('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/test_labels.txt', 'r') 
for line in file.readlines()[1:]: 
    label = line.split()[1]
    y_test.append(label)
unique_train = np.unique(np.array(y_train))
unique_test = np.unique(np.array(y_test))
print(unique_train)
print(unique_test)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(24,8))
plt.title('Histogram of unprocessed classes for the training set')
plt.hist(y_train, bins=len(unique_train), rwidth=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(24,8))
plt.title('Histogram of unprocessed classes for the testing set')
plt.hist(y_test, bins=len(unique_test), rwidth=0.7)
plt.show()

**Μετά την ομαδοποίηση των κλάσεων**

In [ ]:
y_train = []
y_test = []

file = open('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train_labels.txt', 'r') 
for line in file.readlines()[1:]: 
    label = line.split()[1]
    if CLASS_MAPPING[label] is not None:
        y_train.append(CLASS_MAPPING[label])
    
file = open('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/test_labels.txt', 'r') 
for line in file.readlines()[1:]: 
    label = line.split()[1]
    if CLASS_MAPPING[label] is not None:
        y_test.append(CLASS_MAPPING[label])
unique_train = np.unique(np.array(y_train))
unique_test = np.unique(np.array(y_test))
print(unique_train)
print(unique_test)

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Histogram of processed classes for the training set')
plt.hist(y_train, bins=len(unique_train), rwidth=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Histogram of processed classes for the testing set')
plt.hist(y_train, bins=len(unique_test), rwidth=0.7)
plt.show()

### Βήμα 5

In [ ]:

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class PadPackedSequence(nn.Module):
    def __init__(self):
        """Wrap sequence padding in nn.Module
        Args:
            batch_first (bool, optional): Use batch first representation. Defaults to True.
        """
        super(PadPackedSequence, self).__init__()
        self.batch_first = True
        self.max_length = None

    def forward(self, x):
        """Convert packed sequence to padded sequence
        Args:
            x (torch.nn.utils.rnn.PackedSequence): Packed sequence
        Returns:
            torch.Tensor: Padded sequence
        """
        out, lengths = pad_packed_sequence(
            x, batch_first=self.batch_first, total_length=self.max_length  # type: ignore
        )
        lengths = lengths.to(out.device)
        return out, lengths  # type: ignore


class PackSequence(nn.Module):
    def __init__(self):
        """Wrap sequence packing in nn.Module
        Args:
            batch_first (bool, optional): Use batch first representation. Defaults to True.
        """
        super(PackSequence, self).__init__()
        self.batch_first = True

    def forward(self, x, lengths):
        """Pack a padded sequence and sort lengths
        Args:
            x (torch.Tensor): Padded tensor
            lengths (torch.Tensor): Original lengths befor padding
        Returns:
            Tuple[torch.nn.utils.rnn.PackedSequence, torch.Tensor]: (packed sequence, sorted lengths)
        """
        lengths = lengths.to("cpu")
        out = pack_padded_sequence(
            x, lengths, batch_first=self.batch_first, enforce_sorted=False
        )

        return out

In [ ]:
class LSTMBackbone(nn.Module):
    def __init__(
        self,
        input_dim,
        rnn_size=128,
        num_layers=1,
        bidirectional=False,
        dropout=0.1,
    ):
        super(LSTMBackbone, self).__init__()
        self.bidirectional = bidirectional
        self.feature_size = rnn_size * 2 if self.bidirectional else rnn_size

        self.input_dim = input_dim
        self.rnn_size = rnn_size
        self.num_layers = num_layers
        self.hidden_size = rnn_size
        self.pack = PackSequence()
        self.unpack = PadPackedSequence()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=rnn_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=self.bidirectional,
            dropout=dropout,
        )
        self.drop = nn.Dropout(dropout)
    def forward(self, x, lengths):
        """LSTM forward
        Args:
            x (torch.Tensor):
                [B, S, F] Batch size x sequence length x feature size
                padded inputs
            lengths (torch.tensor):
                [B] Original lengths of each padded sequence in the batch
        Returns:
            torch.Tensor:
                [B, H] Batch size x hidden size lstm last timestep outputs
                2 x hidden_size if bidirectional
        """
        packed = self.pack(x, lengths)
        output, _ = self.lstm(packed)
        output, lengths = self.unpack(output)
        output = self.drop(output)

        rnn_all_outputs, last_timestep = self._final_output(output, lengths)
        # Use the last_timestep for classification / regression
        # Alternatively rnn_all_outputs can be used with an attention mechanism
        return last_timestep
    def _merge_bi(self, forward, backward):
        """Merge forward and backward states
        Args:
            forward (torch.Tensor): [B, L, H] Forward states
            backward (torch.Tensor): [B, L, H] Backward states
        Returns:
            torch.Tensor: [B, L, 2*H] Merged forward and backward states
        """
        return torch.cat((forward, backward), dim=-1)

    def _final_output(self, out, lengths):
        """Create RNN ouputs
        Collect last hidden state for forward and backward states
        Code adapted from https://stackoverflow.com/a/50950188
        Args:
            out (torch.Tensor): [B, L, num_directions * H] RNN outputs
            lengths (torch.Tensor): [B] Original sequence lengths
        Returns:
            Tuple[torch.Tensor, torch.Tensor]: (
                merged forward and backward states [B, L, H] or [B, L, 2*H],
                merged last forward and backward state [B, H] or [B, 2*H]
            )
        """

        if not self.bidirectional:
            return out, self._select_last_unpadded(out, lengths)

        forward, backward = (out[..., : self.hidden_size], out[..., self.hidden_size :])
        # Last backward corresponds to first token
        last_backward_out = backward[:, 0, :]
        # Last forward for real length or seq (unpadded tokens)
        last_forward_out = self._select_last_unpadded(forward, lengths)
        out = self._merge_bi(forward, backward)

        return out, self._merge_bi(last_forward_out, last_backward_out)

    def _select_last_unpadded(self, out, lengths):
        """Get the last timestep before padding starts
        Args:
            out (torch.Tensor): [B, L, H] Fprward states
            lengths (torch.Tensor): [B] Original sequence lengths
        Returns:
            torch.Tensor: [B, H] Features for last sequence timestep
        """
        gather_dim = 1  # Batch first
        gather_idx = (
            (lengths - 1)  # -1 to convert to indices
            .unsqueeze(1)  # (B) -> (B, 1)
            .expand((-1, self.hidden_size))  # (B, 1) -> (B, H)
            # (B, 1, H) if batch_first else (1, B, H)
            .unsqueeze(gather_dim)
        )
        # Last forward for real length or seq (unpadded tokens)
        last_out = out.gather(gather_dim, gather_idx).squeeze(gather_dim)

        return last_out

In [ ]:
def train_LSTM(train_set, val_set, input_size):
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Hyper-parameters
    input_size = input_size
    rnn_size = 100
    num_layers = 2
    num_epochs = 100
    bidirectional = True
    
    min_val_loss = np.Inf
    earlystop = 10
    no_improve_in_val = 0

    model = LSTMBackbone(input_size, rnn_size, num_layers, bidirectional=bidirectional)
    model.to(device)
    model = model.double()
    best_model = model
    # Loss and optimizer
    loss_function  = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        # Οbtain the model's device ID
        device = next(model.parameters()).device
        for index, batch in enumerate(train_set):
            features, labels, lengths = batch

            # Move the batch tensors to the right device
            features = features.to(device)
            labels = labels.to(device)

            # Step 1 - zero the gradients
            # Remember that PyTorch accumulates gradients.
            # We need to clear them out before each batch!
            optimizer.zero_grad()
            # Step 2 - forward pass: y' = model(x)
            y_preds = model(features, lengths)
            # Step 3 - compute loss: L = loss_function(y, y')
            loss = loss_function(y_preds, labels)
            # Step 4 - backward pass: compute gradient wrt model parameters
            loss.backward()
            # Step 5 - update weights
            optimizer.step()
            # Accumulate loss in a variable.
            train_loss += loss.data.item()
        train_loss = train_loss / index
        model.eval()
        validation_loss = 0.0
        y_pred = []  # the predicted labels
        y = []  # the gold labels
        # Obtain the model's device ID
        device = next(model.parameters()).device
        # IMPORTANT: in evaluation mode, we don't want to keep the gradients
        # so we do everything under torch.no_grad()
        with torch.no_grad():
            for index, batch in enumerate(val_set):
                # Get the inputs (batch)
                features, labels, lengths = batch
                # Step 1 - move the batch tensors to the right device
                features = features.to(device)
                labels = labels.to(device)
                # Step 2 - forward pass: y' = model(x)
                y_preds = model(features, lengths)  # EX9
                # Step 3 - compute loss: L = loss_function(y, y')
                # We compute the loss only for inspection (compare train/test loss)
                # because we do not actually backpropagate in test time
                loss = loss_function(y_preds, labels)
                # Step 4 - make predictions (class = argmax of posteriors)
                y_preds_arg = torch.argmax(y_preds, dim=1)
                # Step 5 - collect the predictions, gold labels and batch loss
                y_pred.append(y_preds_arg.cpu().numpy())
                y.append(labels.cpu().numpy())
                # Accumulate loss in a variable
                validation_loss += loss.data.item()
        validation_loss = validation_loss/index
        if validation_loss <= min_val_loss:
            no_improve_in_val = 0
            best_model = model
        else:
            no_improve_in_val += 1
        if no_improve_in_val == earlystop:
            break
        if epoch%5 == 0 or epoch == 49: 
            print("For Epoch",epoch, ':', 'Train loss =', train_loss, 'and Validation loss =', validation_loss)
            print()
    return best_model
   

In [ ]:
from sklearn.metrics import classification_report

def evaluate(test_loader, model):
    model.eval()

    y_pred = []  # the predicted labels
    y = []  # the gold labels
    # Obtain the model's device ID
    device = next(model.parameters()).device
    with torch.no_grad():
        for index, batch in enumerate(test_loader):
            # Get the inputs (batch)
            inputs, labels, lengths = batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            y_preds = model(inputs, lengths) 
            y_preds_arg = torch.argmax(y_preds, dim=1)
            y_pred.append(y_preds_arg.cpu().numpy())
            y.append(labels.cpu().numpy())
    print(classification_report(y, y_pred))

### Βήμα 6

**Εκπαίδευση με τα Spectrograms**

In [ ]:
model = train_LSTM(train_loader_mel, val_loader_mel, 128)

In [ ]:
evaluate(test_loader_mel, model)

**Εκπαίδευση με Beat-Synced Spectograms**

In [ ]:
model = train_LSTM(train_loader_beat_mel, val_loader_beat_mel, 128)

In [ ]:
evaluate(test_loader_beat_mel, model)

**Εκπαίδευση με Beat-Synced Χρωμογραφήματα**

In [ ]:
model = train_LSTM(train_loader_beat_chroma, val_loader_beat_chroma, 12)

In [ ]:
evaluate(test_loader_beat_chroma, model)

**Εκπαίδευση με Χρωμογραφήματα και Φασματογραφήματα**

In [ ]:
model = train_LSTM(train_loader, val_loader, 140)

In [ ]:
evaluate(test_loader, model)

### Βήμα 7

In [ ]:
# Define some useful functions for loading spectrograms and chromagrams

def read_fused_spectrogram(spectrogram_file):
    spectrogram = np.load(spectrogram_file)
    return spectrogram.T


def read_mel_spectrogram(spectrogram_file):
    spectrogram = np.load(spectrogram_file)[:128]
    return spectrogram.T

    
def read_chromagram(spectrogram_file):
    spectrogram = np.load(spectrogram_file)[128:]
    return spectrogram.T

In [ ]:
# Define an encoder for the labels

class LabelTransformer(LabelEncoder):
    def inverse(self, y):
        try:
            return super(LabelTransformer, self).inverse_transform(y)
        except:
            return super(LabelTransformer, self).inverse_transform([y])

    def transform(self, y):
        try:
            return super(LabelTransformer, self).transform(y)
        except:
            return super(LabelTransformer, self).transform([y])

In [ ]:
# Define a PaddingTransformer in order to convert all input sequences to the same length

class PaddingTransform(object):
    def __init__(self, max_length, padding_value=0):
        self.max_length = max_length
        self.padding_value = padding_value

    def __call__(self, s):
        if len(s) == self.max_length:
            return s

        if len(s) > self.max_length:
            return s[:self.max_length]

        if len(s) < self.max_length:
            s1 = copy.deepcopy(s)
            pad = np.zeros((self.max_length - s.shape[0], s.shape[1]), dtype=np.float32)
            s1 = np.vstack((s1, pad))
            return s1

In [ ]:
# Define useful parameters that are the same for all the models.

num_mel = 128
num_chroma = 12
n_classes = 10

In [ ]:
class SpectrogramDataset(Dataset):
    def __init__(self, path, class_mapping=None, train=True, max_length=-1, read_spec_fn=read_fused_spectrogram):
        t = 'train' if train else 'test'
        p = os.path.join(path, t)
        self.index = os.path.join(path, "{}_labels.txt".format(t))
        self.files, labels = self.get_files_labels(self.index, class_mapping)
        self.feats = [read_spec_fn(os.path.join(p, f)) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length)
        self.label_transformer = LabelTransformer()
        if isinstance(labels, (list, tuple)):
            self.labels = np.array(self.label_transformer.fit_transform(labels)).astype('int64')

    def get_files_labels(self, txt, class_mapping):
        with open(txt, 'r') as fd:
            lines = [l.rstrip().split('\t') for l in fd.readlines()[1:]]
        files, labels = [], []
        for l in lines:
            label = l[1]
            if class_mapping:
                label = class_mapping[l[1]]
            if not label:
                continue
            # Kaggle automatically unzips the npy.gz format so this hack is needed
            _id = l[0].split('.')[0]
            npy_file = '{}.fused.full.npy'.format(_id)
            files.append(npy_file)
            labels.append(label)
        return files, labels
    

    def __getitem__(self, item):
        # Return a tuple in the form (padded_feats, label, length)
        l = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], l

    def __len__(self):
        return len(self.labels)

**Mel Spectograms**

In [ ]:
mel_specs = SpectrogramDataset(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/',
         train=True,
         class_mapping=CLASS_MAPPING,
         max_length=-1,
         read_spec_fn=read_mel_spectrogram)

train_loader_mel, val_loader_mel = torch_train_val_split(mel_specs, 32 ,32, val_size=.33)

In [ ]:
test_mel = SpectrogramDataset(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/',
         train=False,
         class_mapping=CLASS_MAPPING,
         max_length=1293,
         read_spec_fn=read_mel_spectrogram)

test_loader_mel = DataLoader(test_mel, batch_size=32)

In [ ]:
# Training Per Epoch.

def train_dataset(_epoch, dataloader, model, loss_function, optimizer):
    model.train()
    running_loss = 0.0
    device = next(model.parameters()).device
    
    for index, batch in enumerate(dataloader, 1):
        # Get the inputs (batch)
        inputs, labels, lengths = batch

        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        y_preds = model(inputs, lengths)
        
        loss = loss_function(y_preds, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.data.item()

    return running_loss / index

In [ ]:
#Evaluate Per Epoch
def eval_dataset(dataloader, model, loss_function):
    model.eval()
    running_loss = 0.0

    y_pred = []  
    y = []

    device = next(model.parameters()).device

    with torch.no_grad():
        for index, batch in enumerate(dataloader, 1):
            inputs, labels, lengths = batch

            inputs = inputs.to(device)
            labels = labels.to(device)
            y_preds = model(inputs, lengths)

            loss = loss_function(y_preds, labels)
            y_preds_arg = torch.argmax(y_preds, dim=1)

            y_pred.append(y_preds_arg.cpu().numpy())
            y.append(labels.cpu().numpy())

            # Accumulate loss in a variable
            running_loss += loss.data.item()

    return running_loss / index, (y, y_pred)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 5)
        self.conv1_bn = nn.BatchNorm2d(3)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(3, 6, 5)
        self.conv2_bn = nn.BatchNorm2d(6)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(6, 8, 3)
        self.conv3_bn = nn.BatchNorm2d(8)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(8, 12, 3)
        self.conv4_bn = nn.BatchNorm2d(12)
        self.pool4 = nn.MaxPool2d(2, 2)
                
        self.fc1 = nn.Linear(4680, 128)
        
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x, lengths):
        x = x.view(x.size(0), 1, x.size(1), x.size(2))
        
        x = self.pool1(F.relu( self.conv1_bn(self.conv1(x))))
        x = self.pool2(F.relu( self.conv2_bn(self.conv2(x))))
        x = self.pool3(F.relu( self.conv3_bn(self.conv3(x))))
        x = self.pool4(F.relu( self.conv4_bn(self.conv4(x))))
        
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [ ]:
EPOCHS = 50
model = ConvNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.double()
model.to(device)
loss_function = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
n_epochs_stop = 10
min_val_loss = 1000
epochs_no_improve = 0

for epoch in range(EPOCHS):
    # Train the model for one epoch
    train_dataset(epoch, train_loader_mel, model, loss_function, optimizer)
    
    # Evaluate the performance of the model, on both data sets
    train_loss, (y_train_gold, y_train_pred) = eval_dataset(train_loader_mel, model, loss_function)

    val_loss, (y_val_gold, y_val_pred) = eval_dataset(val_loader_mel, model, loss_function)
    
    if val_loss < min_val_loss:
        # Save the model
        torch.save(model, "./mel_cnn")
        epochs_no_improve = 0
        min_val_loss = val_loss
    else:
        epochs_no_improve += 1
            
    if epochs_no_improve == n_epochs_stop:
        print('Early stopping!')
        break
        
    y_train_true = np.concatenate( y_train_gold, axis=0 )
    y_val_true = np.concatenate( y_val_gold, axis=0 )
    y_train_pred = np.concatenate( y_train_pred, axis=0 )
    y_val_pred = np.concatenate( y_val_pred, axis=0 )
    print("Epoch ", epoch)
    print("Train loss: ", train_loss)
    print("Validation loss: ", val_loss)
    
# Save the model for future evaluation  
torch.save(model, './mel_cnn')


In [ ]:
model = torch.load('./mel_cnn')
model.eval()
_, (y_test_gold, y_test_pred) = eval_dataset(test_loader_mel, model, loss_function)
y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print(classification_report(y_test_true, y_test_pred))

## Βήμα 8

In [ ]:
class MultitaskDataset(Dataset):
    def __init__(self, path, max_length=-1, read_spec_fn=read_fused_spectrogram, label_type='energy'):
        p = os.path.join(path, 'train')
        self.label_type = label_type
        self.index = os.path.join(path, "train_labels.txt")
        self.files, labels = self.get_files_labels(self.index)
        self.feats = [read_spec_fn(os.path.join(p, f)) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length) 
        if isinstance(labels, (list, tuple)):
            self.labels = np.array(labels)

    def get_files_labels(self, txt):
        with open(txt, 'r') as fd:
            lines = [l.split(',') for l in fd.readlines()[1:]]
        files, labels = [], []
        for l in lines:
            if self.label_type == 'valence':
                labels.append(float(l[1]))
            elif self.label_type == 'energy':
                labels.append(float(l[2]))
            elif self.label_type == 'danceability':
                labels.append(float(l[3].strip("\n")))
            else:
                labels.append([float(l[1]), float(l[2]), float(l[3].strip("\n"))])
            # Kaggle automatically unzips the npy.gz format so this hack is needed
            _id = l[0]
            npy_file = '{}.fused.full.npy'.format(_id)
            files.append(npy_file)
        return files, labels
    

    def __getitem__(self, item):
        # Return a tuple in the form (padded_feats, valence, energy, danceability, length)
        l = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], l

    def __len__(self):
        return len(self.labels)

In [ ]:
specs_multi_valence = MultitaskDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset/',
         max_length=1293,
         label_type='valence',
         read_spec_fn=read_mel_spectrogram)

train_loader_valence , val_loader_valence = torch_train_val_split(specs_multi_valence, 32 ,32, val_size=.33)

In [ ]:
specs_multi_energy = MultitaskDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset/',
         max_length=1293,
         label_type='energy',
         read_spec_fn=read_mel_spectrogram)

train_loader_energy, val_loader_energy = torch_train_val_split(specs_multi_energy, 32 ,32, val_size=.33)

In [ ]:
specs_multi_danceability = MultitaskDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset/',
         max_length=1293,
         label_type='danceability',
         read_spec_fn=read_mel_spectrogram)

train_loader_danceability, val_loader_danceability = torch_train_val_split(specs_multi_danceability, 32 ,32, val_size=.33)

**Beat mulstitask dataset**


In [ ]:
beat_specs_multi_valence = MultitaskDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset_beat/',
         max_length=-1,
         label_type='valence',
         read_spec_fn=read_mel_spectrogram)

beat_train_loader_valence , beat_val_loader_valence = torch_train_val_split(beat_specs_multi_valence, 32 ,32, val_size=.33)

In [ ]:
beat_specs_multi_energy = MultitaskDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset_beat/',
         max_length=-1,
         label_type='energy',
         read_spec_fn=read_mel_spectrogram)

beat_train_loader_energy, beat_val_loader_energy = torch_train_val_split(beat_specs_multi_energy, 32 ,32, val_size=.33)

In [ ]:
beat_specs_multi_danceability = MultitaskDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset_beat/',
         max_length=-1,
         label_type='danceability',
         read_spec_fn=read_mel_spectrogram)

beat_train_loader_danceability, beat_val_loader_danceability = torch_train_val_split(beat_specs_multi_danceability, 32 ,32, val_size=.33)

In [ ]:
class BasicLSTM(nn.Module):
    def __init__(self, input_dim, rnn_size, output_dim, num_layers, bidirectional=False, dropout=0):
        super(BasicLSTM, self).__init__()
        self.bidirectional = bidirectional
        self.rnn_size = rnn_size
        self.feature_size = rnn_size * 2 if self.bidirectional else rnn_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        # Initialize the LSTM, Dropout, Output layers
        
        self.lstm = nn.LSTM(input_dim, self.rnn_size, self.num_layers, bidirectional=self.bidirectional, batch_first=True, dropout=self.dropout)
        self.linear = nn.Linear(self.feature_size, output_dim)

    def forward(self, x, lengths):
        """ 
            x : 3D numpy array of dimension N x L x D
                N: batch index
                L: sequence index
                D: feature index

            lengths: N x 1
         """
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        if self.bidirectional:
            h0 = torch.zeros(self.num_layers*2, x.size(0), self.rnn_size).double().to(device)
            c0 = torch.zeros(self.num_layers*2, x.size(0), self.rnn_size).double().to(device)
        else:
            h0 = torch.zeros(self.num_layers, x.size(0), self.rnn_size).double().to(device)
            c0 = torch.zeros(self.num_layers, x.size(0), self.rnn_size).double().to(device)
            
        # Forward 
        lstm_out, _ = self.lstm(x, (h0, c0))
        last_outputs = self.linear(self.last_timestep(lstm_out, lengths, self.bidirectional))
        return last_outputs.view(-1)

    def last_timestep(self, outputs, lengths, bidirectional=False):
        """
            Returns the last output of the LSTM taking into account the zero padding
        """
        if bidirectional:
            forward, backward = self.split_directions(outputs)
            last_forward = self.last_by_index(forward, lengths)
            last_backward = backward[:, 0, :]
            return torch.cat((last_forward, last_backward), dim=-1)

        else:
            return self.last_by_index(outputs, lengths)

    @staticmethod
    def split_directions(outputs):
        direction_size = int(outputs.size(-1) / 2)
        forward = outputs[:, :, :direction_size]
        backward = outputs[:, :, direction_size:]
        return forward, backward

    @staticmethod
    def last_by_index(outputs, lengths):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        idx = (lengths - 1).view(-1, 1).expand(outputs.size(0),
                                               outputs.size(2)).unsqueeze(1).to(device)
        return outputs.gather(1, idx).squeeze()

In [ ]:
def train_dataset_regression(_epoch, dataloader, model, loss_function, optimizer):
    model.train()
    running_loss = 0.0

    device = next(model.parameters()).device
    
    for index, batch in enumerate(dataloader, 1):
        inputs, labels, lengths = batch

        inputs = inputs.to(device)
        labels = labels.to(device)
            
    
        # clear gradients out before each batch
        optimizer.zero_grad()

        #forward pass
        y_preds = model(inputs, lengths)
        loss = loss_function(y_preds, labels.double())

        #backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.data.item()

    return running_loss / index

In [ ]:
def eval_dataset_regression(dataloader, model, loss_function):
    model.eval()
    running_loss = 0.0

    y_pred = []  
    y = []  

    device = next(model.parameters()).device

    with torch.no_grad():
        for index, batch in enumerate(dataloader, 1):
            inputs, labels, lengths = batch

            inputs = inputs.to(device)
            labels = labels.to(device)

            #forward pass
            y_preds = model(inputs, lengths)

            #compute loss
            loss = loss_function(y_preds, labels.double())

            y_pred.append(y_preds.cpu().numpy())
            y.append(labels.cpu().numpy())

            running_loss += loss.data.item()

    return running_loss / index, (y, y_pred)

**Valence with LSTM**

In [ ]:
RNN_SIZE = 32
EPOCHS = 20

model = BasicLSTM(num_mel, RNN_SIZE, 1, 1, bidirectional=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.double()
model.to(device)
loss_function = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    train_dataset_regression(epoch, beat_train_loader_valence, model, loss_function, optimizer)
    
    train_loss, _ = eval_dataset_regression(beat_train_loader_valence, model, loss_function)

    val_loss, _ = eval_dataset_regression(beat_val_loader_valence, model, loss_function)
    
    if epoch % 5 == 0: 
        print("Epoch ", epoch)
        print("Train loss: ", train_loss)
        print("Validation loss: ", val_loss)
        
torch.save(model, './multitask_lstm_valence')

In [ ]:
model = torch.load('./multitask_lstm_valence')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression(beat_val_loader_valence, model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print("Spearman: ", spearmanr(y_test_true, y_test_pred)[0])

**Energy with LSTM**

In [ ]:
RNN_SIZE = 32
EPOCHS = 20

model = BasicLSTM(num_mel, RNN_SIZE, 1, 1, bidirectional=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.double()
model.to(device)
loss_function = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    # Train the model for one epoch
    train_dataset_regression(epoch, beat_train_loader_energy, model, loss_function, optimizer)
    
    # Evaluate the performance of the model, on both data sets
    train_loss, _ = eval_dataset_regression(beat_train_loader_energy, model, loss_function)

    val_loss, _ = eval_dataset_regression(beat_val_loader_energy, model, loss_function)
    
    if epoch % 5 == 0: 
        print("Epoch %d " %epoch)
        print("Train loss: %f" %train_loss)
        print("Validation loss: %f" %val_loss)
        
torch.save(model, './multitask_lstm_energy')

In [ ]:
model = torch.load('./multitask_lstm_energy')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression(beat_val_loader_energy, model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print("Spearman: %f" %spearmanr(y_test_true, y_test_pred)[0])

**Danceability with LSTM**

In [ ]:
RNN_SIZE = 32
EPOCHS = 20

model = BasicLSTM(num_mel, RNN_SIZE, 1, 1, bidirectional=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.double()
model.to(device)
loss_function = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    train_dataset_regression(epoch, beat_train_loader_danceability, model, loss_function, optimizer)
    
    train_loss, _ = eval_dataset_regression(beat_train_loader_danceability, model, loss_function)

    val_loss, _ = eval_dataset_regression(beat_val_loader_danceability, model, loss_function)
    
    if epoch % 5 == 0: 
        print("Epoch %d " %epoch)
        print("Train loss: %f" %train_loss)
        print("Validation loss: %f" %val_loss)
        
torch.save(model, './multitask_lstm_danceability')

In [ ]:
model = torch.load('./multitask_lstm_danceability')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression(beat_val_loader_danceability, model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print("Spearman: ", spearmanr(y_test_true, y_test_pred)[0])

In [ ]:
#CNN with regression

class ConvNetMultitask(nn.Module):
    def __init__(self):
        super(ConvNetMultitask, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 5)
        self.conv1_bn = nn.BatchNorm2d(3)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(3, 6, 5)
        self.conv2_bn = nn.BatchNorm2d(6)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(6, 8, 3)
        self.conv3_bn = nn.BatchNorm2d(8)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(8, 12, 3)
        self.conv4_bn = nn.BatchNorm2d(12)
        self.pool4 = nn.MaxPool2d(2, 2)
                
        self.fc1 = nn.Linear(4680, 128)
        
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x, lengths):
        x = x.view(x.size(0), 1, x.size(1), x.size(2))
        
        x = self.pool1(F.relu( self.conv1_bn(self.conv1(x))))
        x = self.pool2(F.relu( self.conv2_bn(self.conv2(x))))
        x = self.pool3(F.relu( self.conv3_bn(self.conv3(x))))
        x = self.pool4(F.relu( self.conv4_bn(self.conv4(x))))
        
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


**Valence with CNN**

In [ ]:
EPOCHS = 20

model = ConvNetMultitask()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.double()
model.to(device)
loss_function = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

for epoch in range(EPOCHS):
    # Train the model for one epoch
    train_dataset_regression(epoch, train_loader_valence, model, loss_function, optimizer)
    
    # Evaluate the performance of the model, on both data sets
    train_loss, (y_train, y_train_pred) = eval_dataset_regression(train_loader_valence, model, loss_function)

    val_loss, (y_val, y_val_pred) = eval_dataset_regression(val_loader_valence, model, loss_function)
    
    if epoch % 5 == 0: 
        print("Epoch %d " %epoch)
        print("Train loss: %f" %train_loss)
        print("Validation loss: %f" %val_loss)
        y_train_true = np.concatenate( y_train, axis=0 )
        y_train_pred = np.concatenate( y_train_pred, axis=0 )
        print("Spearman in train: %f" %spearmanr(y_train_true, y_train_pred)[0])
        y_val_true = np.concatenate( y_val, axis=0 )
        y_val_pred = np.concatenate( y_val_pred, axis=0 )
        print("Spearman in validation: %f" %spearmanr(y_val_true, y_val_pred)[0])
        print()
        
torch.save(model, './multitask_cnn_valence')

In [ ]:
model = torch.load('./multitask_cnn_valence')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression(val_loader_valence, model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print("Spearman: ", spearmanr(y_test_true, y_test_pred)[0])

**Energy with CNN**

In [ ]:
EPOCHS = 20

model = ConvNetMultitask()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.double()
model.to(device)
loss_function = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

for epoch in range(EPOCHS):
    train_dataset_regression(epoch, train_loader_energy, model, loss_function, optimizer)
    
    train_loss, (y_train, y_train_pred) = eval_dataset_regression(train_loader_energy, model, loss_function)

    val_loss, (y_val, y_val_pred) = eval_dataset_regression(val_loader_energy, model, loss_function)
    
    if epoch % 5 == 0: 
        print("Epoch %d " %epoch)
        print("Train loss: %f" %train_loss)
        print("Validation loss: %f" %val_loss)
        y_train_true = np.concatenate( y_train, axis=0 )
        y_train_pred = np.concatenate( y_train_pred, axis=0 )
        print("Spearman in train: %f" %spearmanr(y_train_true, y_train_pred)[0])
        y_val_true = np.concatenate( y_val, axis=0 )
        y_val_pred = np.concatenate( y_val_pred, axis=0 )
        print("Spearman in validation: %f" %spearmanr(y_val_true, y_val_pred)[0])
        print()
        
torch.save(model, './multitask_cnn_energy')

In [ ]:
model = torch.load('./multitask_cnn_energy')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression(val_loader_energy, model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print("Spearman: ", spearmanr(y_test_true, y_test_pred)[0])
print()

**Danceability with CNN**

In [ ]:
EPOCHS = 20

model = ConvNetMultitask()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.double()
model.to(device)
loss_function = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

for epoch in range(EPOCHS):
    train_dataset_regression(epoch, train_loader_danceability, model, loss_function, optimizer)
    
    train_loss, (y_train, y_train_pred) = eval_dataset_regression(train_loader_danceability, model, loss_function)

    val_loss, (y_val, y_val_pred) = eval_dataset_regression(val_loader_danceability, model, loss_function)
    
    if epoch 5 == 0: 
        print("Epoch %d " %epoch)
        print("Train loss: %f" %train_loss)
        print("Validation loss: %f" %val_loss)
        y_train_true = np.concatenate( y_train, axis=0 )
        y_train_pred = np.concatenate( y_train_pred, axis=0 )
        print("Spearman in train: %f" %spearmanr(y_train_true, y_train_pred)[0])
        y_val_true = np.concatenate( y_val, axis=0 )
        y_val_pred = np.concatenate( y_val_pred, axis=0 )
        print("Spearman in validation: ", spearmanr(y_val_true, y_val_pred)[0])
        
torch.save(model, './multitask_cnn_danceability')

In [ ]:
model = torch.load('./multitask_cnn_danceability')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression(val_loader_danceability, model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print("Spearman: ", spearmanr(y_test_true, y_test_pred)[0])

## Βήμα 9

In [ ]:
#κάνουμε load το μοντέλο του βήματος 7
transfer_model = torch.load('./mel_cnn')
print(transfer_model)

In [ ]:
#Οι παράμετροι του αρχικού μοντέλου δεν θα ξανααλλάξουν
for param in transfer_model.parameters():
    param.requires_grad = False

In [ ]:
# Αλλάζουμε το τελευταίο layer για να ταιριάξει με το νέο δίκτυο
transfer_model.fc2 = nn.Linear(128, 1)
print(transfer_model)

In [ ]:
#Εκπαιδεύεται το τελευταίο layer
EPOCHS = 20

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transfer_model.double()
transfer_model.to(DEVICE)
loss_function = nn.MSELoss().to(DEVICE)
optimizer = torch.optim.Adam(transfer_model.parameters(), lr=0.001, weight_decay=1e-5)

for epoch in range(EPOCHS):
    # Train the model for one epoch
    train_dataset_regression(epoch, train_loader_valence, transfer_model, loss_function, optimizer)
    
    # Evaluate the performance of the model, on both data sets
    train_loss, (y_train, y_train_pred) = eval_dataset_regression(train_loader_valence, transfer_model, loss_function)

    val_loss, (y_val, y_val_pred) = eval_dataset_regression(val_loader_valence, transfer_model, loss_function)
    
    if epoch % 5 == 0: 
        print("Epoch %d " %epoch)
        print("Train loss: %f" %train_loss)
        print("Validation loss: %f" %val_loss)
        y_train_true = np.concatenate( y_train, axis=0 )
        y_train_pred = np.concatenate( y_train_pred, axis=0 )
        print("Spearman in train: %f" %spearmanr(y_train_true, y_train_pred)[0])
        y_val_true = np.concatenate( y_val, axis=0 )
        y_val_pred = np.concatenate( y_val_pred, axis=0 )
        print("Spearman in validation: %f" %spearmanr(y_val_true, y_val_pred)[0])

        
torch.save(transfer_model, './multitask_transfer_valence')

In [ ]:
transfer_model = torch.load('./multitask_transfer_valence')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression(val_loader_valence, transfer_model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print("Spearman: %f" %spearmanr(y_test_true, y_test_pred)[0])
print()

In [ ]:
transfer_model = torch.load('./mel_cnn')

for param in transfer_model.parameters():
    param.requires_grad = False
    
transfer_model.fc2 = nn.Linear(128, 1)


EPOCHS = 20

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transfer_model.double()
transfer_model.to(DEVICE)
loss_function = nn.MSELoss().to(DEVICE)
optimizer = torch.optim.Adam(transfer_model.parameters(), lr=0.001, weight_decay=1e-5)

for epoch in range(EPOCHS):
    train_dataset_regression(epoch, train_loader_energy, transfer_model, loss_function, optimizer)
    
    train_loss, (y_train, y_train_pred) = eval_dataset_regression(train_loader_energy, transfer_model, loss_function)

    val_loss, (y_val, y_val_pred) = eval_dataset_regression(val_loader_energy, transfer_model, loss_function)
    
    if epoch % 5 == 0: 
        print("Epoch ", epoch)
        print("Train loss: ", train_loss)
        print("Validation loss: ", val_loss)
        y_train_true = np.concatenate( y_train, axis=0 )
        y_train_pred = np.concatenate( y_train_pred, axis=0 )
        print("Spearman in train: ", spearmanr(y_train_true, y_train_pred)[0])
        y_val_true = np.concatenate( y_val, axis=0 )
        y_val_pred = np.concatenate( y_val_pred, axis=0 )
        print("Spearman in validation: ", spearmanr(y_val_true, y_val_pred)[0])

        
torch.save(transfer_model, './multitask_transfer_energy')

In [ ]:
transfer_model = torch.load('./multitask_transfer_energy')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression(val_loader_energy, transfer_model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print("Spearman: ", spearmanr(y_test_true, y_test_pred)[0])

In [ ]:
transfer_model = torch.load('./mel_cnn')

for param in transfer_model.parameters():
    param.requires_grad = False
    
transfer_model.fc2 = nn.Linear(128, 1)


EPOCHS = 20

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transfer_model.double()
transfer_model.to(DEVICE)
loss_function = nn.MSELoss().to(DEVICE)
optimizer = torch.optim.Adam(transfer_model.parameters(), lr=0.001, weight_decay=1e-5)

for epoch in range(EPOCHS):
    train_dataset_regression(epoch, train_loader_danceability, transfer_model, loss_function, optimizer)
    
    train_loss, (y_train, y_train_pred) = eval_dataset_regression(train_loader_danceability, transfer_model, loss_function)

    val_loss, (y_val, y_val_pred) = eval_dataset_regression(val_loader_danceability, transfer_model, loss_function)
    
    if epoch%(1) == 0: 
        print("Epoch %d " %epoch)
        print("Train loss: %f" %train_loss)
        print("Validation loss: %f" %val_loss)
        y_train_true = np.concatenate( y_train, axis=0 )
        y_train_pred = np.concatenate( y_train_pred, axis=0 )
        print("Spearman in train: %f" %spearmanr(y_train_true, y_train_pred)[0])
        y_val_true = np.concatenate( y_val, axis=0 )
        y_val_pred = np.concatenate( y_val_pred, axis=0 )
        print("Spearman in validation: %f" %spearmanr(y_val_true, y_val_pred)[0])

        
torch.save(transfer_model, './multitask_transfer_danceability')

In [ ]:
transfer_model = torch.load('./multitask_transfer_danceability')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression(val_loader_danceability, transfer_model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )
print("Spearman: ", spearmanr(y_test_true, y_test_pred)[0])

## Βήμα 10

In [ ]:
specs_multi_all = MultitaskDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset/',
         max_length=1293,
         label_type=-1,
         read_spec_fn=read_mel_spectrogram)

train_loader_all, val_loader_all = torch_train_val_split(specs_multi_all, 32 ,32, val_size=.33)

In [ ]:
def train_dataset_regression_multi(_epoch, dataloader, model, loss_function, optimizer):
    model.train()
    running_loss = 0.0

    device = next(model.parameters()).device
    
    for index, batch in enumerate(dataloader, 1):
        inputs, labels, lengths = batch

        inputs = inputs.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()

        # forward pass
        y_preds_val, y_preds_energy, y_preds_dance = model(inputs, lengths)
        
        # Loss
        loss_1 = loss_function(y_preds_val, labels[:, 0].double())
        loss_2 = loss_function(y_preds_energy, labels[:, 1].double())
        loss_3 = loss_function(y_preds_dance, labels[:, 2].double())
        loss = loss_1 + loss_2 + loss_3

        # backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.data.item()

    return running_loss / index

In [ ]:
def eval_dataset_regression_multi(dataloader, model, loss_function):
    model.eval()
    running_loss = 0.0

    y_pred = []
    y = []

    device = next(model.parameters()).device

    with torch.no_grad():
        for index, batch in enumerate(dataloader, 1):
            inputs, labels, lengths = batch

            inputs = inputs.to(device)
            labels = labels.to(device)

            #Forward pass
            y_preds_val, y_preds_energy, y_preds_dance = model(inputs, lengths)

            # Loss = άθροισμα όλων
            loss_1 = loss_function(y_preds_val, labels[:, 0].double())
            loss_2 = loss_function(y_preds_energy, labels[:, 1].double())
            loss_3 = loss_function(y_preds_dance, labels[:, 2].double())
            loss = loss_1 + loss_2 + loss_3
        
            y_pred.append(np.hstack((y_preds_val.cpu().numpy(), y_preds_energy.cpu().numpy(), y_preds_dance.cpu().numpy())))
            y.append(labels.cpu().numpy())

            running_loss += loss.data.item()
    return running_loss / index, (y, y_pred)

In [ ]:
class ConvNetMultitaskLearning(nn.Module):
    def __init__(self):
        super(ConvNetMultitaskLearning, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 3)
        self.conv1_bn = nn.BatchNorm2d(3)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(3, 6, 3)
        self.conv2_bn = nn.BatchNorm2d(6)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(6, 8, 3)
        self.conv3_bn = nn.BatchNorm2d(8)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(8, 12, 3)
        self.conv4_bn = nn.BatchNorm2d(12)
        self.pool4 = nn.MaxPool2d(2, 2)
        
        self.conv5 = nn.Conv2d(12, 16, 3)
        self.conv5_bn = nn.BatchNorm2d(16)
        self.pool5 = nn.MaxPool2d(2, 2)
                
        self.fc1 = nn.Linear(1216, 32)
        
        self.fc_val = nn.Linear(32, 1)
        
        self.fc_energy = nn.Linear(32, 1)
        
        self.fc_dance = nn.Linear(32, 1)

    def forward(self, x, lengths):
        x = x.view(x.size(0), 1, x.size(1), x.size(2))
        
        x = self.pool1(F.relu( self.conv1_bn(self.conv1(x))))
        x = self.pool2(F.relu( self.conv2_bn(self.conv2(x))))
        x = self.pool3(F.relu( self.conv3_bn(self.conv3(x))))
        x = self.pool4(F.relu( self.conv4_bn(self.conv4(x))))
        x = self.pool5(F.relu( self.conv5_bn(self.conv5(x))))
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        
        energy = self.fc_energy(x)
        
        val = self.fc_val(x)
        
        dance = self.fc_dance(x)
        
        return val, energy, dance

In [ ]:
EPOCHS = 50

model = ConvNetMultitaskLearning()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.double()
model.to(device)
loss_function = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
n_epochs_stop = 10
min_val_loss = 1000
epochs_no_improve = 0

for epoch in range(EPOCHS):
    train_dataset_regression_multi(epoch, train_loader_all, model, loss_function, optimizer)
    
    train_loss, (y_train, y_train_pred) = eval_dataset_regression_multi(train_loader_all, model, loss_function)

    val_loss, (y_val, y_val_pred) = eval_dataset_regression_multi(val_loader_all, model, loss_function)
    
    if val_loss < min_val_loss:
        torch.save(model, './multitask_learning')
        epochs_no_improve = 0
        min_val_loss = val_loss
    else:
        epochs_no_improve += 1
            
    if epochs_no_improve == n_epochs_stop:
        print('Early stopping!')
        break
    if epoch%(1) == 0: 
        print("Epoch ", epoch)
        print("Train loss: ", train_loss)
        print("Validation loss: ", val_loss)
        y_train_true = np.concatenate( y_train, axis=0 )
        y_train_pred = np.concatenate( y_train_pred, axis=0 )

        print("Spearman in train - valence: ", spearmanr(y_train_true[:,0], y_train_pred[:,0])[0])
        print("Spearman in train - energy: ", spearmanr(y_train_true[:,1], y_train_pred[:,1])[0])
        print("Spearman in train - dance: ", spearmanr(y_train_true[:,2], y_train_pred[:,2])[0])
        
        y_val_true = np.concatenate( y_val, axis=0 )
        y_val_pred = np.concatenate( y_val_pred, axis=0 )
        print("Spearman in validation - valence: ", spearmanr(y_val_true[:,0], y_val_pred[:,0])[0])
        print("Spearman in validation - energy: ", spearmanr(y_val_true[:,1], y_val_pred[:,1])[0])
        print("Spearman in validation - dance: ", spearmanr(y_val_true[:,2], y_val_pred[:,2])[0])
        

In [ ]:
model = torch.load('./multitask_learning')

test_loss, (y_test_gold, y_test_pred) = eval_dataset_regression_multi(val_loader_all, model, loss_function)

y_test_true = np.concatenate( y_test_gold, axis=0 )
y_test_pred = np.concatenate( y_test_pred, axis=0 )

print(y_test_true.shape)

print("Spearman: ", spearmanr(y_test_true[:,0], y_test_pred[:,0])[0])
print("Spearman: ", spearmanr(y_test_true[:,1], y_test_pred[:,1])[0])
print("Spearman: ", spearmanr(y_test_true[:,2], y_test_pred[:,2])[0])